# Search AiiDA Database for Isotherms

In [ ]:
%aiida
import collections
import sys
import ipywidgets as ipw
from IPython.display import clear_output

CifData = DataFactory('cif')

In [ ]:
def discrete_cmap(colors):
    """Return discrete colormap for matplotlib.
    
    :param: list of RGB colors
    """

    color_size = len(colors)
    base = plt.cm.get_cmap('cubehelix')
    cmap_name = "discrete_" + str(color_size)
    return base.from_list(cmap_name, colors, color_size)

def plot(x, y, colors, uuids, ids, title=None, xlabel=None, ylabel=None, clr_label=None):
    import bokeh.plotting as bpl
    import bokeh.models as bmd
    from bokeh.palettes import Viridis256
    from bokeh.io import output_notebook, show
    output_notebook(hide_banner=True)

    cmap = bmd.LinearColorMapper(palette=Viridis256, low=min(colors), high=max(colors))
    
    hover = bmd.HoverTool(tooltips=[
        ("structure id", "@id"),
    ])
    
    fig = bpl.figure(
        toolbar_location=None,
        title=title,
        x_axis_label=xlabel,
        y_axis_label=ylabel,
        tools=['tap', 'zoom_in', 'zoom_out', 'pan', hover], 
        output_backend='webgl',
    )

    source = bmd.ColumnDataSource(data=dict(x=x, y=y, uuid=uuids, id=ids, color=colors))
    fig.circle('x', 'y', size=10, source=source, fill_color={'field':'color', 'transform':cmap})
    
    if len(x) > 1:
        fig.add_layout(bmd.ColorBar(color_mapper=cmap, title=clr_label, location=(0, 0)), 'right')
    
    taptool = fig.select(type=bmd.TapTool)
    url="{}?uuid=@uuid".format( './show.ipynb')
    taptool.callback = bmd.OpenURL(url=url)
    
    show(fig)

In [ ]:
def search(_=None):
    """Query AiiDA database."""
    
    plot_info.value = "Searching..."
    
    filters = {}
    
    for label, obj in sliders.items():
        if obj.enabled:
            filters['attributes.'+obj.aiida_property] = {'and':[{'>=':obj.slider.value[0]}, {'<':obj.slider.value[1]}]}
    

    projections = ['attributes.'+inp_x.value.aiida_property,
                   'attributes.'+inp_y.value.aiida_property,
                   'attributes.'+inp_clr.value.aiida_property]
    

    with output:
        clear_output()
        
        if len(set(projections)) != len(projections):
            plot_info.value = "Please select different quantities for X, Y and Color"
            return

        qb = QueryBuilder()
        qb.append(Str, filters={'attributes.value': molecule.value})
        qb.append(WorkChainNode, project=['uuid'], with_incoming=Str)
        qb.append(Dict, filters=filters, project=projections, with_incoming=WorkChainNode)
        qb.append(CifData, project=['id'], with_outgoing=WorkChainNode)
        results = [k for k in qb.all() if None not in k]
        nresults = len(results)
    
        if nresults == 0:
            plot_info.value = "No results found."
            return

        plot_info.value = "{} results found. Plotting...".format(nresults)

        uuids, x, y, clrs, ids = zip(*results)
        
        plot(
            x=list(map(float, x)),
            y=list(map(float, y)),
            colors=list(map(float, clrs)),
            uuids=list(map(str, uuids)),
            ids=list(map(str, ids)),
            title="{} vs {}".format(inp_y.label, inp_x.label),
            xlabel=inp_x.value.description,
            ylabel=inp_y.value.description,
            clr_label=inp_clr.value.description)
   
        plot_info.value = "Plotted {} results.".format(nresults)

In [ ]:
from traitlets import Bool, link, observe

class FilteredProperty(ipw.HBox):
    enabled = Bool(False)
    def __init__(self, value=None, label='', aiida_property='', unit='', min=0, max=1, step=0.05, **kwargs):
        self.label = label
        self.description = "{} [{}]".format(label, unit)
        self.aiida_property = aiida_property
        self.slider = ipw.FloatRangeSlider(description=self.description,
                                           min=min,
                                           max=max,
                                           value=value or (min, max),
                                           step=0.05,
                                           disabled=True,
                                           layout={"width": "100%"},
                                           style={"description_width":"220px"}
                                          )
        
        self._enabled = ipw.Checkbox(value=False,
                                     description='Apply filter',
                                     layout={"width": "initial"},
                                     disabled=False,
                                     indent=False)        
        link((self, 'enabled'), (self._enabled, 'value'))
        super().__init__( children=[self.slider, self._enabled], layout={"width": "100%"}, **kwargs)

    @observe('enabled')
    def _enable_or_disable(self, _=None):
        self.slider.disabled = not self.enabled

In [ ]:
sliders = collections.OrderedDict([
    ('Density', FilteredProperty(label="Density", aiida_property='Density', unit="g/cm^3", min=0.0, max=20.0)),
    ('Henry coefficient', FilteredProperty(label="Henry coefficient",  aiida_property='henry_coefficient_average', unit='mol/kg/Pa', min=0.0, max=1)),
    ('Adsorption energy widom', FilteredProperty(label="Adsorption energy widom", aiida_property='adsorption_energy_widom_average', unit='kJ/mol', min=-1e3, max=0.0)),
    ('Pore accesible volume', FilteredProperty(label='Pore accesible volume', aiida_property='POAV_cm^3/g', unit='cm^3/g', min=0.0, max=100.0)),
    ('Pore non-accesible volume', FilteredProperty(label='Pore non-accesible volume', aiida_property='PONAV_cm^3/g', unit='cm^3/g', min=0.0, max=100.0)),
    ('Unit cell volume', FilteredProperty(label='Unit cell volume', aiida_property='Unitcell_volume', unit='A^3', min=0.0, max=1e5)),
    ('Estimated saturation loading', FilteredProperty(label='Estimated saturation loading', aiida_property='Estimated_saturation_loading', unit='mol/kg', min=0, max=1000.0)),
])


inp_x = ipw.Dropdown(
    options = sliders,
    label='Density',
    description='X:',
)

inp_y = ipw.Dropdown(
    options = sliders,
    label='Pore accesible volume',
    description='Y:',
)

inp_clr = ipw.Dropdown(
    options = sliders,
    label='Henry coefficient',
    description='Color:',
)


molecule = ipw.Dropdown(
    options=[
        ("CO2", "co2"),
        ("CH4", "ch4"),
        ("N2",  "n2"),
        ("H2O", "h2o"),
        ("H2",  "h2"),
        ("O2",  "o2"),
    ],
    description="Guest molecule",
    style={'description_width':'initial'},
    layout={'margin': '20px 0px 20px 0px'},
)

search_button = ipw.Button(description="Plot", layout={'margin': '0px 20px 0px 0px'},)
search_button.on_click(search)
output = ipw.Output()
plot_info = ipw.HTML("")

display(
    ipw.VBox(
        list(sliders.values()) + [ipw.HBox([inp_x, inp_y, inp_clr]),
                                  molecule,
                                  ipw.HBox([search_button, plot_info]),
                                  output,
                                 ]
    )
)